In [ ]:
import pandas as pd

In [ ]:
# load the childhood obesity dataset from the raw excel
childhood_obesity_df = pd.read_excel("data/target_data_obesity/MSOA_Ward_LA_Obesity.xlsx", sheet_name="2011-12_2013-14", skiprows=1)

In [ ]:
first_five_column_values = childhood_obesity_df.head(2).tail(1).values.tolist()[0][:5]

In [ ]:
current_column_values_minus_first_five= childhood_obesity_df.columns[5:].tolist()
twenty_11_columns = current_column_values_minus_first_five[:12]
twenty_11_columns_reception = ["reception_4yrs_5yrs_old_"+x for x in twenty_11_columns[:6]]
twenty_11_columns_year_6 = ["year_6_10yrs_11yrs_old_"+x for x in twenty_11_columns[6:]]
columns_to_keep = first_five_column_values + twenty_11_columns_reception + twenty_11_columns_year_6
new_col_headers = first_five_column_values + current_column_values_minus_first_five
new_col_headers

In [ ]:
childhood_obesity_df.columns = columns_to_keep + current_column_values_minus_first_five[12:]

In [ ]:
childhood_obesity_df = childhood_obesity_df.tail(-2)


In [ ]:
idx_cols = first_five_column_values
childhood_obesity_int_df = childhood_obesity_df.melt(id_vars = idx_cols)
childhood_obesity_int_df = childhood_obesity_int_df[~childhood_obesity_int_df.variable.str.contains("Unnamed")]
childhood_obesity_int_df = childhood_obesity_int_df[~childhood_obesity_int_df["value"].isna()]
childhood_obesity_int_df = childhood_obesity_int_df[~childhood_obesity_int_df["LA code"].isna()]
childhood_obesity_int_df = childhood_obesity_int_df[childhood_obesity_int_df["Geog Level"]=="MSOA"]
# drop anything without year 6 or recption in the column name
childhood_obesity_int_df = childhood_obesity_int_df[childhood_obesity_int_df["variable"].isin(columns_to_keep)]
# only keep columns with the words obese or measured
childhood_obesity_int_df = childhood_obesity_int_df[(childhood_obesity_int_df["variable"].str.contains("obese"))
                                                    |(childhood_obesity_int_df["variable"].str.contains("measured"))]

# clean up the column names and remove the .1 etc..
childhood_obesity_int_df["variable"] = childhood_obesity_int_df["variable"].str.replace(".1","")

# pivot this back to normal now
clean_childhood_obesity_df = childhood_obesity_int_df.pivot(index = idx_cols, columns = "variable", values="value").reset_index()
clean_childhood_obesity_df
# childhood_obesity_int_df

# Write to disk

In [ ]:
%%time
columns_to_drop = ['Geog Level',
                     'Area name',
                     'LA code',
                     'LA name'
                  ]
output_df = clean_childhood_obesity_df.drop(columns=columns_to_drop).rename(columns={"Code":"geography code"})
output_df.to_csv("data/staged_files/s1_clean_target_variable_childhood_obesity_dataset.csv", index=False)